In [21]:
using DynamicPolynomials, SparseArrays, MatrixMarket

println("***Problem setting***")

#n=25





# random quadratic objective function f

#=
A=sprand(n, n, 0.5)
for i in 1:n
    for j=i:n
        if i == j
            A[j,j]=1
        else
            A[i,j]=sign(A[j,i])
            A[j,i]=A[i,j]
        end
    end
end
A=Matrix(A)
=#


#include("./mat_stability_size$(n).jl")
A = MatrixMarket.mmread("GD02_a.mtx")

n=size(A,1)
for i in 1:n
    A[i,i]=1
end
@polyvar x[1:n]# variables

println("Number of variables: n=",n)
println("====================")

#f=sum(A[i,j]*x[i]*x[j] for i=1:n for j=1:n)*1.0
f=0.0
for j in 1:n
    for i in j:n
        if i==j
            f+=A[i,j]*x[i]*x[j]
        else
            f+=2*A[i,j]*x[i]*x[j]
        end
    end
end
            
g=Vector{Polynomial{true,Float64}}([])
#type of coefficients of each polynomial must be float

m=length(g)
println("Number of inequality constraints: m=",m)
println("====================")            

# unit sphere constraint
h=[1.0-sum(x)] #type of coefficients of each polynomial must be float

l=length(h)

println("Number of equality constraints: l=",l)
println("====================")

***Problem setting***
Number of variables: n=23
Number of inequality constraints: m=0
Number of equality constraints: l=1


In [24]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);


k=1

println("Relaxation order: k=",k)

s=25

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek",comp_opt_sol=false)

Relaxation order: k=1
Block size: s=25


Maximal matrix size:24
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 2600            
  Cones                  : 0               
  Scalar variables       : 2325            
  Matrix variables       : 24              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :             

(-3.124039519537318e-7, Float64[])

In [23]:
1/opt_val1

-7.027549513616335e7

In [19]:
include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=false);


k=0

println("Relaxation order: k=",k)

s=12

println("Block size: s=",s)

@time opt_val1,opt_sol=InterruptedRelax.RelaxDense(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,
    lmon_f,supp_f,coe_f,dg,dh,k,s,solver="Mosek")

Relaxation order: k=0
Block size: s=12


Maximal matrix size:12
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 300             
  Cones                  : 0               
  Scalar variables       : 301             
  Matrix variables       : 13              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 23
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   

(0.06666661251018555, Float64[])

In [20]:
1/opt_val1

15.00001218521815

In [15]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

using TSSOS

k=1

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterruptedRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 24
************************TSSOS************************


TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[23]
[1]
------------------------------------------------------
Assembling the SDP...
There are 300 affine constraints.
SDP assembling time: 0.000475511 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 300             
  Cones                  : 0               
  Scalar variables       : 25              
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1      

In [16]:
1/opt

-0.8223245712550831

In [6]:
using SparseArrays

include("../src/InterruptedRelax.jl")
using .InterruptedRelax

n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh=InterruptedRelax.get_info(x,f,g,h,sparse=true);

#InterruptedRelax.save_info_sparsePOP(randx,u,n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f,dg,dh)

#include("./sparsePOP3.jl")

using TSSOS

k=2

println("Maximal matrix size: ",binomial(n+k,k))

vars,pop=InterruptedRelax.get_POP(n,m,l,lmon_g,supp_g,coe_g,lmon_h,supp_h,coe_h,lmon_f,supp_f,coe_f)

@time opt,sol,data=TSSOS.cs_tssos_first(pop,vars,k,numeq=l,CS=false,TS=false,solver="Mosek");

Maximal matrix size: 300
************************TSSOS************************


TSSOS is launching...
------------------------------------------------------
The clique sizes of varibles:
[23]
[1]
------------------------------------------------------
Assembling the SDP...
There are 17550 affine constraints.
SDP assembling time: 0.341633141 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 17550           
  Cones                  : 0               
  Scalar variables       : 301             
  Matrix variables       : 24              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1    

In [7]:
1/opt

13.011034945138055